In [ ]:
import numpy as np
import pandas as pd

In [ ]:
f_path = "C:/Users/Dell/Documents/FQ304大修人员剂量明细.xlsx"

In [ ]:
data = pd.read_excel(
    f_path,
    sheet_name="数据源",
    usecols=[
        "人员编号",
        "姓名",
        "单位",
        "处室",
        "机组",
        "进入区域",
        "进入时间",
        "离开时间",
        "持续时间(h)",
        "EPD-γ剂量(mSv)"
    ],
    dtype={
        "人员编号": str,
        "进入时间": "datetime64",
        "离开时间": "datetime64"
    }
)
data.head()

In [ ]:
day_number = data.groupby(
    # ["人员编号"]
    # pd.Grouper(key="离开时间", freq="D"),
    [pd.Grouper(key="离开时间", freq="D"), "人员编号", "姓名", "处室", "单位"]
).agg({"EPD-γ剂量(mSv)": np.sum, "人员编号": np.count_nonzero})
day_number

In [ ]:
day_number.info()

In [ ]:
day_number.query("处室 == '保健物理处'")